In [ ]:
import pandas as pd 
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
scallop = pd.read_csv("scallop.csv", usecols = ["latitude", "longitude","tot.catch"])
scallop.head()

Divided data by longitude

In [ ]:
data = scallop.copy()
data["tot.catch"] = np.log(scallop["tot.catch"] + 1)
# choose longitude more than -72 as testing data
train = data[data["longitude"]< -72]
test = data[data["longitude"]>= -72]

In [ ]:
plt.scatter(train["longitude"], train["latitude"], alpha=0.5,
            c=train["tot.catch"], cmap='viridis')
plt.scatter(test["longitude"], test["latitude"], alpha=0.5,
            c=test["tot.catch"], cmap='viridis',  marker='x')
plt.colorbar()
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.show()

In [ ]:
X_2D_train = np.c_[train["longitude"],  train["latitude"]]
Y_2D_train = np.array(train["tot.catch"])

X_2D_test = np.c_[test["longitude"],  test["latitude"]]
Y_2D_test = np.array(test["tot.catch"])

rx = np.arange(min(data["longitude"]), max(data["longitude"]), 0.06)
ry = np.arange(min(data["latitude"]), max(data["latitude"]), 0.06)
gx, gy = np.meshgrid(rx, ry)


X_2D = np.c_[gx.ravel(), gy.ravel()]
len(X_2D)

In [ ]:
def plot_gp_2D(gx, gy, mu,sd, X_train, Y_train, X_test, Y_test):
    z_min = min(min(mu), min(Y_train))
    z_max = max(max(mu), max(Y_train))
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    c = ax1.pcolormesh(gx, gy,mu.reshape(gx.shape), vmin = z_min,vmax = z_max, alpha=0.2,cmap='viridis')
    ax1.scatter(X_train[:,0], X_train[:,1], alpha=0.2, c=Y_train, vmin = z_min,vmax = z_max,cmap='viridis')
    ax1.scatter(X_test[:,0], X_test[:,1], marker='x', alpha=0.2, c=Y_test, cmap='viridis')
    fig.colorbar(c, ax = ax1)
    ax1.set_xlabel("longitude")
    ax1.set_ylabel("latitude")
    ax1.set_title("Posterior mean")
    
    
    
    c = ax2.pcolormesh(gx, gy, sd.reshape(gx.shape), alpha=0.2, cmap='viridis')
    fig.colorbar(c, ax = ax2)
    ax2.set_xlabel("longitude")
    ax2.set_ylabel("latitude")
    ax2.set_title("Posterior sd")

# Gaussian Process ---- Hyperparameters 
the Kernel Squared Exponential (SE) as equation 2.16 in the text book:
<center>$kernel\_SE = \sigma_f^2 exp(-\frac{1}{2l^2}|x_i - x_j|^2)$</center>

The hyperparameters we are interested in are $\sigma_n, l, \sigma_f$:
1. $\sigma_f$: the scale of the output values (the overall variance of the process).
2. l: the scale at which distances are measured among inputs (the distance from which on two points will be uncorrelated) 
3. $\sigma_n$: the noise

## MCMC using PyMC3

Define the prior of those three parameters (demonstrated by Stan tutorial) :\
https://mc-stan.org/docs/2_22/stan-users-guide/fit-gp-section.html#priors-gp.section
1. l ~ invGamma(5,5)
2. sigma_f ~ Normal(0, 1)
3. sigma_n ~ Normal(0, 1)

Sampling by Nuts


In [ ]:
import pymc3 as pm
import theano
theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"

# hyperparameter priors
number_of_dim = 2
niter =1000
x = X_2D_train
y = Y_2D_train
with pm.Model() as scallop_model_1:
    l = pm.InverseGamma("l", 5, 5)
    sigma_f = pm.Normal("sigma_f", 0, 1)
    

# covariance function and marginal GP
with scallop_model_1:
    K = sigma_f** 2 * pm.gp.cov.ExpQuad(number_of_dim, ls = l)
    gp = pm.gp.Marginal(cov_func=K)
    
# marginal likelihood
with scallop_model_1:
    sigma_n = pm.HalfNormal("sigma_n",1)
    tot_catch = gp.marginal_likelihood("tot_catch", X = x, y = y, noise = sigma_n)
    
# model fitting
with scallop_model_1:
    trace_1 = pm.sample(niter, random_seed=123, progressbar=True, tune=500)

In [ ]:
pm.traceplot(trace_1)

In [ ]:
pm.summary(trace_1)

In [ ]:
X_new = X_2D
# y = Y_2D_train
with scallop_model_1:
    scallop_pred_noisy = gp.conditional("scallop_pred_noisy",X_new,pred_noise = True)
    scallop_samples = pm.sample_posterior_predictive(trace_1, vars = [scallop_pred_noisy],samples=50)

In [ ]:
mu_1 = np.zeros(len(X_new))
sd_1 = np.zeros(len(X_new))

for i in range(0,len(X_new)):
    mu_1[i] = np.mean(scallop_samples["scallop_pred_noisy"][:,i])
    sd_1[i] = np.std(scallop_samples["scallop_pred_noisy"][:,i])

In [ ]:
plot_gp_2D(gx, gy, mu_1,sd_1, X_2D_train, Y_2D_train, X_2D_test, Y_2D_test)

In [ ]:
# y = Y_2D_train
with scallop_model_1:
    scallop_pred_noisy_test = gp.conditional("scallop_pred_noisy_test",X_2D_test,pred_noise = True)
    scallop_samples_test = pm.sample_posterior_predictive(trace_1, vars = [scallop_pred_noisy_test],samples=50)

mu_test_1 = np.zeros(len(X_2D_test))
sd_test_1 = np.zeros(len(X_2D_test))

for i in range(0,len(X_2D_test)):
    mu_test_1[i] = np.mean(scallop_samples_test["scallop_pred_noisy_test"][:,i])
    mu_test_1[i] = np.std(scallop_samples_test["scallop_pred_noisy_test"][:,i])

In [ ]:
pd.DataFrame({"pred":mu_test_1,
              "true":Y_2D_test,
              "lower": mu_test_1 - 1.96 * sd_test_1,
              "upper": mu_test_1 + 1.96 * sd_test_1})

In [ ]:
RMSE = np.sqrt(np.mean((np.exp(mu_test_1) - np.exp(Y_2D_test))**2))
RMSE

Define the prior of those three parameters :
1. l ~ HalfCauchy(3)
2. sigma_f ~ HalfCauchy(3)
3. sigma_n ~ halfNormal(1)


Sampling by Nuts

In [ ]:
import pymc3 as pm

# hyperparameter priors
number_of_dim = 2
niter =1000
x = X_2D_train
y = Y_2D_train

with pm.Model() as scallop_model_2:
    l = pm.HalfCauchy("l",3)
    sigma_f = pm.HalfCauchy("sigma_f",3)

# covariance function and marginal GP
with scallop_model_2:
    K = sigma_f ** 2 * pm.gp.cov.ExpQuad(number_of_dim, ls = l)
    gp = pm.gp.Marginal(cov_func=K)
    
# marginal likelihood
with scallop_model_2:
    sigma_n = pm.HalfNormal('sigma_n', 1)
    tot_catch = gp.marginal_likelihood("tot_catch", X = x, y = y, noise = sigma_n)
    
# model fitting
with scallop_model_2:
    trace_2 = pm.sample(niter,random_seed=123, progressbar=True, tune=500)



In [ ]:
pm.traceplot(trace_2)

In [ ]:
pm.summary(trace_2)

In [ ]:
X_new = X_2D
# y = Y_2D_train
with scallop_model_2:
    scallop_pred_noisy_2 = gp.conditional("scallop_pred_noisy_2",X_new,pred_noise = True)
    scallop_samples_2 = pm.sample_posterior_predictive(trace_2, vars = [scallop_pred_noisy_2],samples=50)

In [ ]:
mu_2 = np.zeros(len(X_new))
sd_2 = np.zeros(len(X_new))

for i in range(0,len(X_new)):
    mu_2[i] = np.mean(scallop_samples_2["scallop_pred_noisy_2"][:,i])
    sd_2[i] = np.std(scallop_samples_2["scallop_pred_noisy_2"][:,i])

In [ ]:
plot_gp_2D(gx, gy, mu_2,sd_2, X_2D_train, Y_2D_train, X_2D_test, Y_2D_test)

In [ ]:
# y = Y_2D_train
with scallop_model_2:
    scallop_pred_noisy_test_2 = gp.conditional("scallop_pred_noisy_test_2",X_2D_test,pred_noise = True)
    scallop_samples_test_2 = pm.sample_posterior_predictive(trace_2, vars = [scallop_pred_noisy_test_2],samples=50)

In [ ]:
mu_test_2 = np.zeros(len(X_2D_test))
sd_test_2 = np.zeros(len(X_2D_test))

for i in range(0,len(X_2D_test)):
    mu_test_2[i] = np.mean(scallop_samples_test_2["scallop_pred_noisy_test_2"][:,i])
    sd_test_2[i] = np.std(scallop_samples_test_2["scallop_pred_noisy_test_2"][:,i])

In [ ]:
pd.DataFrame({"pred":mu_test_2,
              "true":Y_2D_test,
              "lower": mu_test_2 - 1.96 * sd_test_2,
              "upper": mu_test_2 + 1.96 * sd_test_2})

In [ ]:
RMSE = np.sqrt(np.mean((np.exp(mu_test_2) - np.exp(Y_2D_test))**2))
RMSE

### Inverse Gamma distribution and Cauchy Distribution

In [ ]:
from scipy.stats import invgamma, halfcauchy

In [ ]:
x = np.linspace (0, 10, 200) 
y1 = halfcauchy.pdf(x, 3)
plt.plot(x, y1, "y-", label=(r'$\alpha=5, \beta=1/5$')) 

plt.ylim([0,1])
plt.xlim([0,10])
plt.legend()
plt.show()

In [ ]:
x = np.linspace (4, 7, 200) 
y1 = invgamma.pdf(x, a=5, loc=5)
plt.plot(x, y1, "y-", label=(r'$\alpha=5, \beta=1/5$')) 

plt.ylim([0,5])
plt.xlim([4,7])
plt.legend()
plt.show()

In [ ]:
scallop_model_1.name = "scallop_model_1"
scallop_model_2.name = "scallop_model_2"
df_comp_LOO = pm.compare({scallop_model_1: trace_1, scallop_model_2: trace_2}, ic='LOO')
df_comp_LOO

## MLE
Posterior
$mean = K(X_s,X)[K(X,X) + \sigma_n^2 *I]^{-1}y$\
$cov = K(X_s,X_s) - K(X_s,X)[K(X,X) + \sigma_n^2 *I]^{-1} K(X,X_s)$\
$logp(y|X, \theta) = -\frac{1}{2}y^T(K + \sigma_n^2)^{-1}y - 1/2log(K + \sigma_n^2) - \frac{n}{2}log(2\pi)$



In [ ]:
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
from numpy.linalg import inv

def kernel(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def posterior_predictive(kernel, X_s, X_train, Y_train,l=1.0, sigma_f=1.0, sigma_y=1e-8):
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s_ = kernel(X_s, X_train, l, sigma_f)
    K__s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f)
    K_inv = inv(K)
    
    # Equation (4)
    mu_s = K_s_.dot(K_inv).dot(Y_train)

    # Equation (5)
    cov_s = K_ss - K_s_.dot(K_inv).dot(K__s)

    return mu_s, cov_s

def nll_fn(X_train, Y_train):
    
    def neg_log_lik(theta):
        # theta[0] = l
        # theta[1] = sigma_f
        # theta[2] = sigma_n
        K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + theta[2]**2 * np.eye(len(X_train))
        return 0.5 * np.log(det(K)) + \
               0.5 * Y_train.T.dot(inv(K)).dot(Y_train) + \
               0.5 * len(X_train) * np.log(2*np.pi)
    return neg_log_lik

res = minimize(nll_fn(X_2D_train, Y_2D_train), [1, 1, 1], 
               bounds=((1e-5, None), (1e-5, None), (1e-5, None)),
               method='L-BFGS-B')


In [ ]:
l_opt, sigma_f_opt, sigma_n_opt = res.x
l_opt, sigma_f_opt, sigma_n_opt

In [ ]:
# Compute the posterior predictive statistics with optimized kernel parameters and plot the results
mu_s, cov_s = posterior_predictive(kernel, X_2D, X_2D_train, Y_2D_train, l=l_opt, sigma_f=sigma_f_opt, sigma_y=sigma_n_opt)
plot_gp_2D(gx, gy, mu_s,np.sqrt(np.diag(cov_s)), X_2D_train, Y_2D_train, X_2D_test, Y_2D_test)

RMSE

In [ ]:
mu_s_test, cov_s_test = posterior_predictive(kernel, X_2D_test, X_2D_train, Y_2D_train, l=l_opt, sigma_f=sigma_f_opt, sigma_y=sigma_n_opt)
RMSE = np.sqrt(np.mean((np.exp(mu_s_test) - np.exp(Y_2D_test))**2))
RMSE

In [ ]:
mu_s_test, cov_s_test = posterior_predictive(kernel, X_2D_test, X_2D_train, Y_2D_train, l=1, sigma_f=1, sigma_y=0.2)
RMSE = np.sqrt(np.mean((np.exp(mu_s_test) - np.exp(Y_2D_test))**2))
RMSE